In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D # type: ignore
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Embedding # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.losses import SparseCategoricalCrossentropy # type: ignore

N = Number of samples (documents -- not in the above)  
T = number of "words" (in text) or steps: **sequence length**  
D = number of input features  
M = **size of hidden vector** (number of hidden units)  
K = number of output units (classes/clasification or values/regression)  

In [2]:
N = 1 # one document
T = 10
D = 3
K = 2
X = np.random.randn(N, T, D)

In [3]:
M = 5
i = Input(shape=(T, D))
x = SimpleRNN(M)(i)
x = Dense(K)(x)
model = Model(i, x)

In [6]:
# Get the output
Yhat = model.predict(X)
print(Yhat) # Can we replicate this?

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.8921652  0.41974723]]


In [5]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57 (228.00 B)

 Trainable params: 57 (228.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
# check weights
model.layers[1].get_weights()

[array([[-0.27443564,  0.35013777,  0.5039118 ,  0.709315  ,  0.84286755],
        [-0.47468916, -0.4397845 ,  0.3993643 ,  0.14419335, -0.5735241 ],
        [-0.5177796 , -0.3348847 , -0.8468714 ,  0.5441082 ,  0.82040733]],
       dtype=float32),
 array([[ 0.3841474 , -0.09217948, -0.90066624,  0.06285825,  0.16965589],
        [-0.04645918, -0.9176449 ,  0.06801343,  0.352915  , -0.16307767],
        [ 0.71323925, -0.16694133,  0.40592816, -0.1799806 ,  0.51599395],
        [-0.31167296,  0.1444778 ,  0.02579516,  0.64490956,  0.6822113 ],
        [-0.49439064, -0.31731266, -0.13686377, -0.650537  ,  0.46147642]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

Above are three arrays.  
The shapes (below) should make sense.  
First output is input > hidden  
Second output is hidden > hidden  
Third output is bias terms (vector of length M) 

In [ ]:
a, b, c = model.layers[1].get_weights()
print(a.shape, b.shape, c.shape)
# a: (D, M) - input weights
# b: (M, M) - recurrent weights
# c: (M,) - bias

(3, 5) (5, 5) (5,)


In [13]:
Wx, Wh, bh = model.layers[1].get_weights()
Wo, b0 = model.layers[2].get_weights() # output weights

In [ ]:
# manual RNN calculation
h_last = np.zeros(M) # initial hidden state
x = X[0] # the one and only sample
Yhats = [] # output sequence

for t in range(T):
    h = np.tanh(x[t].dot(Wx) + h_last.dot(Wh) + bh)
    y = h.dot(Wo) + b0 # only interesting ni the last iteration
    Yhats.append(y)

    # assign h to h_last
    h_last = h
# check the output
print(Yhats[-1]) # last output; checks with previous output

[0.89216507 0.41974727]
